* see: https://python.langchain.com/docs/use_cases/question_answering/

In [1]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# two possible vector store
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

# removed OpenAI, using HF
from langchain.embeddings import HuggingFaceEmbeddings

from langchain import hub

# removed OpenAI, using OCI GenAI
import oci

# oci_llm is in a local file
from oci_llm import OCIGenAILLM

from langchain.schema.runnable import RunnablePassthrough

In [2]:
# to enable some debugging
DEBUG = False

In [3]:
# read OCI config to connect to OCI with API key
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file("~/.oci/config", CONFIG_PROFILE)

# OCI GenAI endpoint (for now Chicago)
ENDPOINT = "https://generativeai.aiservice.us-chicago-1.oci.oraclecloud.com"

# check the config to access to api keys
if DEBUG:
    print(config)

#### Loading the document

In [4]:
BLOG_POST = "https://python.langchain.com/docs/get_started/introduction"
loader = WebBaseLoader(BLOG_POST)

data = loader.load()

#### Splitting the document in chunks

In [5]:
CHUNK_SIZE = 512

text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=0)

splits = text_splitter.split_documents(data)

In [6]:
print(f"We have {len(splits)} splits...")

We have 8 splits...


In [7]:
# have a look at a single split
if DEBUG:
    print(splits[1])

#### Embeddings and Vectore Store

In [8]:
# We have substituted OpenAI with HF
EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"

model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}


hf = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_NAME, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# using Chroma or FAISS as Vector store
# vectorstore = Chroma.from_documents(documents=splits,
#                                    embedding=hf)
vectorstore = FAISS.from_documents(documents=splits, embedding=hf)

retriever = vectorstore.as_retriever()

#### Define the prompt structure

In [ ]:
rag_prompt = hub.pull("rlm/rag-prompt")

#### Define the LLM

In [ ]:
COMPARTMENT_OCID = "ocid1.compartment.oc1..aaaaaaaag2cpni5qj6li5ny6ehuahhepbpveopobooayqfeudqygdtfe6h3a"

llm = OCIGenAILLM(
    temperature=1,
    config=config,
    compartment_id=COMPARTMENT_OCID,
    endpoint=ENDPOINT,
    debug=DEBUG,
)

#### Define the (Lang)Chain

In [ ]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | rag_prompt | llm

#### Process the question

In [ ]:
# a list of possible questions
QUESTION1 = "What is the best architecture for an LLM?"
QUESTION2 = "What is LangChain?"

In [ ]:
%%time
response = rag_chain.invoke(QUESTION2)

print("The response:")
print(response)
print()

#### Explore the vectore store

In [ ]:
# Retrieve relevant splits for any question using similarity search.

# This is simply "top K" retrieval where we select documents based on embedding similarity to the query.

TOP_K = 5

docs = vectorstore.similarity_search(QUESTION2, k=TOP_K)

len(docs)

In [ ]:
for doc in docs:
    print(doc.page_content)
    print()